In [1]:
data_Allgene = read.csv('/cellar/users/a53206221/Data/Expression/TCGA_TP_allgene.csv')
data = read.csv('/cellar/users/a53206221/Data/PQ3RO1/SCNADf_v3.tsv',sep='\t')
data_Allgene <- merge(data[,c('patient_id','SCNA')],data_Allgene,by="patient_id")
UPR_list = c('CCL2',
 'FKBP14',
 'ACADVL',
 'ADD1',
 'EDEM1',
 'HSP90B1',
 'CUL7',
 'WIPI1',
 'PLA2G4B',
 'TLN1',
 'KLHDC3',
 'SEC31A',
 'XBP1',
 'HERPUD1',
 'DCTN1',
 'SRPRB',
 'HSPA5',
 'DDX11',
 'TATDN2',
 'SYVN1',
 'DNAJB11',
 'EXTL1',
 'ATP6V0D1',
 'HDGF',
 'EXTL2',
 'CTDSP2',
 'DDIT3',
 'SHC1',
 'TSPYL2',
 'TPP1',
 'PDIA5',
 'EXTL3',
 'ARFGAP1',
 'DNAJB9',
 'SULT1A3',
 'DNAJC3',
 'SERP1',
 'IGFBP1',
 'YIF1A',
 'ZBTB17',
 'LMNA',
 'WFS1',
 'KDELR3',
 'CXCL8',
 'CALR',
 'HYOU1',
 'GSK3A',
 'ASNS',
 'PREB',
 'SSR1',
 'CXXC1',
 'MYDGF',
 'PPP2R5B',
 'GOSR2',
 'ATF4',
 'GFPT1',
 'PDIA6',
 'ATF3')

In [2]:
diseaseList = c('STAD',
 'HNSC',
 'LIHC',
 'LUSC',
 'COAD',
 'THYM',
 'LUAD',
 'KIRC',
 'KIRP',
 'LGG',
 'THCA',
 'PAAD',
 'CHOL',
 'BRCA',
 'BLCA',
 'SARC',
 'PRAD',
 'GBM',
 'PCPG',
 'SKCM',
 'OV',
 'TGCT',
 'READ',
 'UCEC',
 'MESO',
 'CESC',
 'KICH',
 'DLBC',
 'ACC',
 'UCS',
 'ESCA',
 'UVM')

for (disease in diseaseList){

    tmp = data_Allgene[data_Allgene$disease == disease,]
    datC1  = tmp[tmp$SCNA <= quantile(x = tmp$SCNA,probs = .3),]
    datC2  = tmp[tmp$SCNA >= quantile(x = tmp$SCNA,probs = .7),]

    # D
    Pathway = UPR_list
    tmp = data_Allgene[data_Allgene$disease == disease ,]
    datC1  = tmp[tmp$SCNA <= quantile(x = tmp$SCNA,probs = .3),]
    datC2  = tmp[tmp$SCNA >= quantile(x = tmp$SCNA,probs = .7),]

    beta1 = 4 #user defined parameter for soft thresholding
    AdjMatC1<-sign(cor(datC1[,as.character(Pathway)],method="spearman"))*(cor(datC1[,as.character(Pathway)],method="spearman"))^2
    AdjMatC2<-sign(cor(datC2[,as.character(Pathway)],method="spearman"))*(cor(datC2[,as.character(Pathway)],method="spearman"))^2

    diag(AdjMatC1)<-0
    diag(AdjMatC2)<-0

    D = (abs(AdjMatC1-AdjMatC2)/2)^(beta1/2)

    # compute permutation
    ### permutation and plot
    datC1 = datC1[,as.character(Pathway)]
    datC2 = datC2[,as.character(Pathway)]

    permutations<-NULL
    for (i in 1:1000)
    {
       permutations<-rbind(permutations,sample(1:(nrow(datC1)+nrow(datC2)),nrow(datC1)))
    }

    # combine
    d<-rbind(datC1,datC2)
    Df_permut=data.frame(matrix(0, ncol = length(UPR_list), nrow = length(UPR_list),))
    rownames(Df_permut)<-rownames(D)
    colnames(Df_permut)<-colnames(D)
    # calculate the D matrix in permutation

    for (i in 1:1000)
        {
        perC1=d[c(permutations[i,]),]
        perC2=d[ ! c(1:(max(permutations))) %in% permutations[i,],]
        AdjMatC1<-sign(cor(perC1,method="spearman"))*(cor(perC1,method="spearman"))^2
        AdjMatC2<-sign(cor(perC2,method="spearman"))*(cor(perC2,method="spearman"))^2

        diag(AdjMatC1)<-0
        diag(AdjMatC2)<-0
        
        D_permut = (abs(AdjMatC1-AdjMatC2)/2)^(beta1/2)

        tmp = D_permut > D
        tmp=tmp+0
        Df_permut=Df_permut+tmp
    }
    # changed directory
    # saved the updated gene sets in the new directory below
    Dir = paste('~/Data/PQ3RO1/difcoexpUpdate.7.6.2020/',disease,'.permut.beta4.csv', sep = "", collapse = NULL)
    write.csv(data.frame(Df_permut),file = Dir)
    }